In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from numpy.random import randn
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;
from sklearn import preprocessing
import math
from sklearn.metrics import mean_squared_error


//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("train.csv")

In [3]:
train["area"] = train["slotheight"]*train["slotwidth"] 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]

In [4]:
X_train.loc[ X_train['slotprice'] <= 0, 'slotprice'] = 0
X_train.loc[(X_train['slotprice'] > 0) & (X_train['slotprice'] <= 10), 'slotprice'] = 1
X_train.loc[(X_train['slotprice'] > 10) & (X_train['slotprice'] <= 50), 'slotprice'] = 2
X_train.loc[(X_train['slotprice'] > 50) & (X_train['slotprice'] <= 100), 'slotprice'] = 3
X_train.loc[(X_train['slotprice'] > 100), 'slotprice'] = 4


//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [5]:
X_train.area.value_counts().sort_index()
X_train.loc[ X_train['area'] <= 20000, 'area'] = 0
X_train.loc[(X_train['area'] > 20000) & (X_train['area'] <=40000 ), 'area'] = 1
X_train.loc[(X_train['area'] > 40000) & (X_train['area'] <= 60000), 'area'] = 2
X_train.loc[(X_train['area'] > 60000) & (X_train['area'] <= 80000), 'area'] = 3
X_train.loc[(X_train['area'] > 80000) & (X_train['area'] <= 100000), 'area'] = 4
X_train.loc[(X_train['area'] > 100000), 'area'] = 5


//anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [6]:
X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,slotprice
0,1,14,1458,2,1,1,0,1
1,2,20,3476,2,3,1,0,1
2,5,9,1458,1,4,0,1,0
3,2,20,3427,2,3,0,0,4
4,6,9,3476,1,4,0,1,0


In [7]:
X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["adexchange"]=X_train.adexchange.astype("str")
X_train["slotvisibility"]=X_train.slotvisibility.astype("str")
X_train["slotformat"]=X_train.slotformat.astype("str")
X_train["hour"]=X_train.hour.astype("str")


X_train=pd.get_dummies(X_train)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pa

In [8]:
logreg = LogisticRegression(penalty='l2', dual=False, tol=0.0001)
logreg.fit(X_train, Y_train)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

99.920000000000002

In [9]:
Y_pred = logreg.predict_proba(X_train)
Y_pred_log = Y_pred[:,1]
Y_pred_log

array([ 0.00079687,  0.00066757,  0.00086359, ...,  0.00034695,
        0.00071373,  0.00091072])

In [10]:
import math
from sklearn.metrics import mean_squared_error
MSE = mean_squared_error(Y_pred_log ,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027433795364052205


In [11]:
validation = pd.read_csv("validation.csv")

## Gradient

In [12]:
train["area"] = train["slotheight"]*train["slotwidth"] 

city = pd.DataFrame(train.city.value_counts()/train.shape[0]).index
frequent = pd.DataFrame(train.city.value_counts()/train.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
train=pd.merge(train,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_train = train[feature]
Y_train = train["click"]

X_train.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,1,14,1458,2,28080,1,0,0.019417,5
1,2,20,3476,2,62500,1,0,0.013130,5
2,5,9,1458,1,94080,0,1,0.039022,0
3,2,20,3427,2,65520,0,0,0.004477,162
4,6,9,3476,1,85500,0,1,0.001714,0


In [13]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_train["slotvisibility"] = label_encoder.fit_transform(X_train["slotvisibility"])
X_train["slotformat"] = label_encoder.fit_transform(X_train["slotformat"])
X_train["adexchange"] = label_encoder.fit_transform(X_train["adexchange"])

X_train["advertiser"]=X_train.advertiser.astype("str")
X_train["weekday"]=X_train.weekday.astype("str")
X_train["hour"]=X_train.hour.astype("str")

X_train=pd.get_dummies(X_train)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [14]:
GBR = GradientBoostingClassifier()
GBR.fit(X_train, Y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [15]:
Y_pred_ = GBR.predict_proba(X_train)
Y_pred_gbr = Y_pred_[:,1]
Y_pred_gbr

array([ 0.00076917,  0.00041292,  0.00108197, ...,  0.00024882,
        0.00047198,  0.00082217])

In [16]:
MSE = mean_squared_error(Y_pred_gbr,Y_train)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.027383548816209902


### Performance Matrix --- Train
#### Base bid = 100

In [17]:
train["pCTR_gbr"] = Y_pred_gbr

avgCTR = Y_pred_gbr.mean()

param_gbr= Y_pred_gbr/avgCTR

train['new_bidprice_gbr'] = 100*param_gbr

statistic=pd.DataFrame()
new_train = train[train["new_bidprice_gbr"]>=train.payprice]
new_train["total_cost"] = new_train["payprice"].cumsum()
budget_1 = new_train[new_train["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_train[new_train["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_train[new_train["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_train[new_train["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_train[new_train["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,395,24999.972,0.078175,63.291068,49.477756
1/2,197,12499.958,0.077903,63.451563,49.430589
1/4 (6250),92,6249.983,0.072807,67.934598,49.460938
1/8,51,3124.936,0.080531,61.273255,49.343692
1/16,22,1562.432,0.069396,71.019636,49.284966


### Validation

In [18]:
validation["area"] = validation["slotheight"]*validation["slotwidth"] 

city = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).index
frequent = pd.DataFrame(validation.city.value_counts()/validation.shape[0]).values
r = pd.DataFrame()
r["city"] = city
r["frequent"] = frequent
validation=pd.merge(validation,r,on='city',how='left') 

feature = ["weekday","hour","advertiser","adexchange","area","slotvisibility","slotformat","frequent",
          "slotprice"]
X_validation = validation[feature]
Y_validation = validation["click"]

X_validation.head()

,weekday,hour,advertiser,adexchange,area,slotvisibility,slotformat,frequent,slotprice
0,5,14,3427,1,75000,0,1,0.002068,0
1,3,14,2997,null,16000,FirstView,Na,0.039229,118
2,1,13,3427,3,90000,0,0,0.003676,70
3,5,18,3427,3,90000,0,0,0.010175,70
4,5,14,3386,2,94080,2,0,0.021628,5


In [19]:
np.random.seed(12)

# Initialize label encoder
label_encoder = preprocessing.LabelEncoder()

X_validation["slotvisibility"] = label_encoder.fit_transform(X_validation["slotvisibility"])
X_validation["slotformat"] = label_encoder.fit_transform(X_validation["slotformat"])
X_validation["adexchange"] = label_encoder.fit_transform(X_validation["adexchange"])


X_validation["advertiser"]=X_validation.advertiser.astype("str")
X_validation["weekday"]=X_validation.weekday.astype("str")
X_validation["hour"]=X_validation.hour.astype("str")

X_validation=pd.get_dummies(X_validation)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

In [20]:
Y_pred_validation_gbr = GBR.predict_proba(X_validation)
Y_pred_validation_gbr = Y_pred_validation_gbr [:,1]
MSE = mean_squared_error(Y_pred_validation_gbr,Y_validation)
RMSE = math.sqrt(MSE)
print("RMSE: " + str(RMSE) )

RMSE: 0.02751658128834068


### Performace Matrix--validaiton
#### Base bid = 100

In [21]:
validation["pCTR_gbr"] = Y_pred_validation_gbr

avgCTR = Y_pred_validation_gbr.mean()

param_validation_gbr= Y_pred_validation_gbr/avgCTR

validation['new_bidprice_gbr'] = 100*param_validation_gbr

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full = statistic_3.append(statistic_16)

statistic_full.index = ["full","1/2","1/4(6250)","1/8","1/16"]
statistic_full

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,143,9917.983,0.071288,69.356524,49.442823
1/2,143,9917.983,0.071288,69.356524,49.442823
1/4(6250),94,6249.948,0.074236,66.488809,49.358711
1/8,56,3124.918,0.088473,55.802107,49.369913
1/16,31,1562.483,0.097937,50.402677,49.362872


#### base price = 70

In [22]:
validation['new_bidprice_gbr'] = 70*param_validation_gbr

statistic=pd.DataFrame()
new_validation = validation[validation["new_bidprice_gbr"]>=validation.payprice]
new_validation["total_cost"] = new_validation["payprice"].cumsum()
budget_1 = new_validation[new_validation["total_cost"]/1000<=25000]
statistic["Click"] = [budget_1[budget_1.click==1].click.sum()]
statistic["Spend"] = budget_1.payprice.sum()/1000
statistic["CTR%"] = statistic["Click"]/budget_1.shape[0]*100
statistic["CPC"] = statistic["Spend"]/statistic["Click"]
statistic["CPM"] = statistic["Spend"] /budget_1.shape[0]* 1000

statistic_2=pd.DataFrame()
budget_2 = new_validation[new_validation["total_cost"]/1000<=25000*1/2]
statistic_2["Click"] = [budget_2[budget_2.click==1].click.sum()]
statistic_2["Spend"] = budget_2.payprice.sum()/1000
statistic_2["CTR%"] = statistic_2["Click"]/budget_2.shape[0]*100
statistic_2["CPC"] = statistic_2["Spend"]/statistic_2["Click"]
statistic_2["CPM"] = statistic_2["Spend"] /budget_2.shape[0]* 1000

statistic_4=pd.DataFrame()
budget_4 = new_validation[new_validation["total_cost"]/1000<=25000/4]
statistic_4["Click"] = [budget_4[budget_4.click==1].click.sum()]
statistic_4["Spend"] = budget_4.payprice.sum()/1000
statistic_4["CTR%"] = statistic_4["Click"]/budget_4.shape[0]*100
statistic_4["CPC"] = statistic_4["Spend"]/statistic_4["Click"]
statistic_4["CPM"] = statistic_4["Spend"] /budget_4.shape[0]* 1000

statistic_8=pd.DataFrame()
budget_8 = new_validation[new_validation["total_cost"]/1000<=25000*1/8]
statistic_8["Click"] = [budget_8[budget_8.click==1].click.sum()]
statistic_8["Spend"] = budget_8.payprice.sum()/1000
statistic_8["CTR%"] = statistic_8["Click"]/budget_8.shape[0]*100
statistic_8["CPC"] = statistic_8["Spend"]/statistic_8["Click"]
statistic_8["CPM"] = statistic_8["Spend"] /budget_8.shape[0]* 1000

statistic_16=pd.DataFrame()
budget_16 = new_validation[new_validation["total_cost"]/1000<=25000*1/16]
statistic_16["Click"] = [budget_16[budget_16.click==1].click.sum()]
statistic_16["Spend"] = budget_16.payprice.sum()/1000
statistic_16["CTR%"] = statistic_16["Click"]/budget_16.shape[0]*100
statistic_16["CPC"] = statistic_16["Spend"]/statistic_16["Click"]
statistic_16["CPM"] = statistic_16["Spend"] /budget_16.shape[0]* 1000


statistic_one = statistic.append(statistic_2)
statistic_two = statistic_one.append(statistic_4)
statistic_3 = statistic_two.append(statistic_8)
statistic_full_val_grb = statistic_3.append(statistic_16)

statistic_full_val_grb.index = ["full","1/2","1/4 (6250)","1/8","1/16"]
statistic_full_val_grb

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Click,Spend,CTR%,CPC,CPM
full,117,6039.890,0.081794,51.622991,42.224296
1/2,117,6039.890,0.081794,51.622991,42.224296
1/4 (6250),117,6039.890,0.081794,51.622991,42.224296
1/8,64,3124.878,0.086241,48.826219,42.108016
1/16,37,1562.369,0.099433,42.226189,41.986751
